In [1]:
import torch
from tqdm import tqdm  
import treescope as ts 
import matplotlib.pyplot as plt
from data import DNADataset
from vae import *
dataset = DNADataset("hf://datasets/Zehui127127/latent-dna-diffusion/sequence.csv")

In [2]:
cycle = 1
model = torch.load(f"model_cycle_{cycle}.pth")
ts.display(model)

/tmp/ipykernel_2562472/3154003660.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f"model_cycle_{cycle}.pth")


FileNotFoundError: [Errno 2] No such file or directory: 'model_cycle_1.pth'

In [12]:
device = torch.device("cuda:0")
model = model.to(device=device)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)
for el in (pbar := tqdm(list(dataloader)[:10])):
    el = el.to(device=device, dtype=torch.float32)
    encoded, _ = model.encode(el)
    decoded = model.decode(encoded)
    quantized = torch.eye(5).to(device)[decoded.argmax(dim=-1)]

    ts.display(
        {"original": el[0].T, "decoded": decoded[0].T, "quantized": quantized[0].T},
        autovisualize=True,
    )

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:03,  2.48it/s]

 20%|██        | 2/10 [00:00<00:02,  3.93it/s]

 30%|███       | 3/10 [00:00<00:01,  4.60it/s]

 40%|████      | 4/10 [00:00<00:01,  5.17it/s]

...

 60%|██████    | 6/10 [00:01<00:00,  5.54it/s]

 70%|███████   | 7/10 [00:01<00:00,  5.62it/s]

 80%|████████  | 8/10 [00:01<00:00,  5.90it/s]

 90%|█████████ | 9/10 [00:01<00:00,  5.40it/s]

100%|██████████| 10/10 [00:01<00:00,  5.16it/s]
